In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from pyspark.ml.evaluation import BinaryClassificationEvaluator as BCE
from pyspark.ml.classification import MultilayerPerceptronClassifier

import week4 as w4

## Introducción

Existen multitud de algoritmos de aprendijaze diferentes y cada uno de ellos puede rendir mejor o peor dependiendo de la tipología y distribución de los datos, la finalidad del modelo, el porcentaje de positivos,...

Por este motivo, es conveniente que, aún y habiendo obtenido un porcentaje de aciertos razonablemente bueno mediante un modelo de regresión logística, valoremos la utilización de otros métodos.

Una de las tipologías de algoritmos más utilizadas para la clasificación de imágenes es la Red Neuronal. En esta sección vamos a utilizar una para intentar obtener una fiabilidad más alta.

## Lectura y manipulación de los datos

Igual como hicimos en la sesión anterior, tenemos que leer y manipular ligeramente los datos.

In [ ]:
df_raw = sqlContext.read.parquet('pca_features.parquet')
df = df_raw.withColumn('label', (df_raw['target'] - 1).astype('double'))['dr7objid', 'pca_features', 'label']

## Modelo de redes neuronales

### Partición Test - Train

In [ ]:
(df_train, df_test) = df.randomSplit([0.7, 0.3], seed=1234)

### Entrenamiento

Primero tenemos que definir la estructura de nuestra red neuronal.

Una red neuronal està compuesta por capas y cada capa por un conjunto de nodos. En pySpark, esta estructura se define a través de una lista en que cada valor representa el número de nodos de la capa.

Así pues:
* la lista [5, 10, 1] define una red neuronal de tres capas con 5, 10 y 1 nodos respecivamente
* la lista [20, 10, 5, 2] define una red de 4 capas con 20, 10, 5 y 2 nodos.

Las redes neuronales tienen 3 tipos de capas:
* de entrada: solo hay una, debe tener tantos nodos como features (atributos) en el dataset y corresponde a la primera posición de la lista.
* ocultas: pueden haber varías y no tienen ninguna restricción en cuanto al número de capas ni de nodos. En general, cuántas más capas ocultas haya, más preciso será el algoritmo y será más costoso de entrenar.
* de salida: solo hay una, debe tener tantos nodos como clases en el dataset y corresponde a la última posición de la lista.

Así pues, para nuestro ejemplo deberemos configurar una red neuronal con una estructura de este tipo
`[64, ..., 2]`

In [ ]:
layers = [64, 16, 8, 2]

Ahora sí, creamos la red neuronal y la entrenamos.

In [ ]:
mlp = MultilayerPerceptronClassifier(labelCol="label",
                                     featuresCol="pca_features", 
                                     maxIter=100, 
                                     layers=layers, 
                                     seed=123)

In [ ]:
mlp_model = mlp.fit(df_train)

### Validación

In [ ]:
pred_df = mlp_model.transform(df_test)

In [ ]:
cm = w4.confussion_matrix(pred_df, 'label', 'prediction').show()

Vemos que el porcentaje de aciertos se ha incrementado con respecto al algoritmo de regresión logística. Ahora tenemos cerca de un 95% de aciertos.

### Ejercicio:

** Lee los datos no etiquetados y clasifícalos utilizando el modelo de Redes Neuronales que hemos entrenado**

In [ ]:
# Lectura de los datos
df_raw_unlabeled = sqlContext.read.parquet('pca_features_unlabeled.parquet')

In [ ]:
# aplicación del modelo a los datos no etiquetados
pred_df_unlabeled = mlp_model.transform(df_raw_unlabeled)

¿Cuàntas galaxias hay de cada tipo según la predicción?

In [ ]:
pred_df_unlabeled.groupBy('prediction').count().show()